In [1]:
from reportlab.lib import colors
from reportlab.lib.units import cm
from Bio.Graphics import GenomeDiagram
from Bio import SeqIO
from Bio.SeqFeature import SeqFeature, FeatureLocation
from IPython.core.display import Image
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
import scipy.stats as stats

In [2]:
ref = "C:/Users/Flora/Desktop/M2 Systèmes Complexes/Stage/Bioinfo/ref/"
data = "C:/Users/Flora/Desktop/M2 Systèmes Complexes/Stage/Bioinfo/LTEE-clone-curated/"
storage = "C:/Users/Flora/Desktop/M2 Systèmes Complexes/Stage/Bioinfo/GenomeDiagram_tests/"

In [3]:
from os import listdir
from os.path import isfile, join

In [4]:
record = SeqIO.read(ref+'REL606.gbk', 'genbank')

In [52]:
def plot_GenomeDiagram(IS_name):
    '''Plots the location of insertions on the genome, with insertions into a given functional category highlighted in red.'''
    
    len_IS = 1500 #could be changed for accuracy but not really necessary
    
    onlyfiles = [f for f in listdir(data) if isfile(join(data, f))] #listing all file names for the .gd files
    files = [] # creating a list that will first contain these names to be called, and later on open .gd files
    file_names = [] # list that will contain names as strings (kept as such for all the analysis)
    readlines_names = [] #list that will contain lists with the lines of each file (obtained via readlines())
    for i in onlyfiles: #fixing an error in the list of files (I don't know why it occurs) and filling the previously defined lists
        if len(i) < 30:
            files.append(i)
            file_names.append(i)
            readlines_names.append(i)
    
    for i in range(len(files)):
        files[i] = open(data+files[i],"r") #the list files becomes a list of open files
        readlines_names[i] = files[i].readlines() # the list readlines_names becomes a list of lists, with each list containing 
        #all the lines from one file                
                         
    insertion_lines = {} # Creating a dictionary to hold all the insertions found in each file
    for file_name in file_names: # Creating one entry per file in the dictionary (an empty list)
        insertion_lines[file_name] = []
    for i in range(len(file_names)): # for each file
        for line in readlines_names[i]: #we look at all the lines
            line = line.split()
            if 'MOB' in line: #if one line corresponds to a mutation linked to a mobile element
                insertion_lines[file_names[i]].append(line) #we add that line to the entry of that file in the dictionary                    
    
    IS_lines={name:[line for line in insertion_lines[name] if line[5] == IS_name] for name in file_names}
    
    gd_diagram = GenomeDiagram.Diagram('REL606') #creating the diagram
    
    pop_names = ["Ara+1", "Ara+2", "Ara+3", "Ara+4", "Ara+5", "Ara+6","Ara-1", "Ara-2", "Ara-3", "Ara-4", "Ara-5", "Ara-6"]
    colors = ['firebrick', 'red', 'gold', 'chartreuse', 'mediumspringgreen', 'darkcyan', 'deepskyblue', 'blue', 'royalblue', 'navy', 'darkorchid', 'violet']
    gd_feature_set_names = ['gd_feature_' + pop for pop in pop_names]
    
    gd_track_for_features_all = gd_diagram.new_track(13, name = 'Annotated Features', scale_ticks = False, scale_color = 'black') #creating a track for all insertions
    gd_feature_set_names_all = gd_track_for_features_all.new_set() #creating a feature set
    
    pop_number = -1
    
    for pop in pop_names:
        
        pop_number += 1
          
        gd_track_for_features = gd_diagram.new_track(pop_number+1, name = 'Annotated Features', scale_ticks = False, scale_color = colors[pop_number]) #creating a track
        gd_feature_set_names[pop_number] = gd_track_for_features.new_set() #creating a feature set
        
        
        for i in range(len(file_names)):
            
            if pop in file_names[i]:

                for IS_line in IS_lines[file_names[i]]: #for each insertion in the dictionary we generated
                    IS_position = int(IS_line[4])
                    position_found = False

                    for feature in record.features[1:]: # we go through the features
                        if feature.type == 'CDS': #we look at coding sequences
                            if 'join' not in str(feature.location): #getting rid of an annoying but unique anomaly in the features
                                start_position = int(str(feature.location).split(':')[0][1:]) #getting the start position of the CDS
                                end_position = int(str(feature.location).split(':')[1][:-4]) #getting the end position of the CDS
                                if IS_position >= start_position and IS_position <= end_position: # if the IS interrupts that CDS
                                    position_found = True
                                    gd_feature_set_names[pop_number].add_feature(feature, color = colors[pop_number], label = False)
                                    #we add that feature to our list of features to plot
                                    gd_feature_set_names_all.add_feature(feature, color = 'black', label = True)

                    if not position_found:
                        new_feature = SeqFeature(FeatureLocation(IS_position,IS_position+len_IS), strand=int(IS_line[6])) 
                        #if we didn't find that feature, we create one and add it in grey (intergenic regions)
                        gd_feature_set_names[pop_number].add_feature(new_feature, color = 'grey', label = False)
                        gd_feature_set_names_all.add_feature(new_feature, color = 'grey', label = True)


    #drawing and saving the diagrams
    gd_diagram.draw(format = 'circular', circular = True, pagesize = (40*cm, 44*cm), start = 0, end = len(record), circle_core = 0.2)
    gd_diagram.write(storage+'test_GenomeDiagram'+'_'+ IS_name + '_' + 'by_pop' + '_with_general_track'+ '.pdf', 'PDF')
    #gd_diagram.write(storage+'test_GenomeDiagram' +'_'+ IS_name+ '_' + 'by_pop' + '_no_labels'+ '.png', 'PNG')
    #somehow there is a color problem resulting in an error for the png so I just stopped generating it, PDFs are fine


In [55]:
plot_GenomeDiagram("IS186")